In [1]:
categories = ["A Cowboy Who Rode the Waves","Car free cities","Cell phones at school","Community service","Distance learning","Does the electoral college work","Driverless cars","Exploring Venus","Facial action coding system","Mandatory extracurricular activities","Phones and driving","Seeking multiple opinions","Summer projects","The Face on Mars"]
category = "The Face on Mars"
input_file = "/scratch1/avuthu/category_split/test_"+category+".csv"
output_file = "/scratch1/avuthu/zero_shot_mistral_category_predictions/predictions_"+category+".txt"

In [2]:
import pandas as pd
import numpy as np
import guidance
import pandas as pd
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, roc_auc_score
import torch

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
model = "mistralai/Mistral-7B-Instruct-v0.1"
#model = "sinking8/mixtral_finetuned"
temp = 0.01
guidance.llm = guidance.llms.Transformers(model, device=device, temperature=temp)

#The function `process_input_text` processes a given text, calls a function `structure_program` on it, and tries to extract an 'answer'
#from the output. If it doesn't find an 'answer', it retries until it does, while managing memory during this process.
# The log probs value for two classes seems to be negative at certain instances and the label that corresponds to the maximum among those is given as an output by guidance.
# To convert the classifier to regressor the abs difference between the log probs are taken assuming that 1 -> Supportive 0-> Unsupportive and passing it to the sigmoid layer.
def process_input_text(prompt_text, structure_program):
    answer = None
    while answer is None:
        out = structure_program(prompt_text=prompt_text)
        try:
            answer = out['answer']
        except KeyError:
            print("Key 'answer' not found in the output. Retrying...")
            #print(out.variables())
            import gc
            gc.collect()
            import torch
            torch.cuda.empty_cache()
    return answer

cuda:0


/home1/avuthu/.local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home1/avuthu/.local/lib/python3.9/site-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
Loading checkpoint shards: 100%|██████████| 2/2 [02:03<00:00, 61.80s/it]


In [3]:
import pandas as pd
data = pd.read_csv(input_file)
input_prompts = data['text'].tolist()
true_labels=data["label"].tolist()
print(len(data))

1893


In [4]:
#Can only run 1000 queries at a time. After that need to restart the kernel on CARC
start=0
stop=1000

In [5]:
labels = []

import torch
import warnings
def run_code(model, prompt, temp):
    structure_program = guidance(prompt)    
    for i, t in enumerate(input_prompts[start:stop]):
        label = process_input_text(t,structure_program)
        labels.append(label)
    with open(output_file, 'a') as file:
    #Write each label followed by a newline character
        for label in labels:
            file.write(label + '\n')
    file.close() 

In [6]:
#Run the main script
import time
start_time=time.time()
model = "mistralai/Mistral-7B-Instruct-v0.1" #huggingface model
temperature = 0.02 #the lower the more deterministic
prompt = '''Is the given following prompt AI generated? Answer with a single word, either "yes" or "no".
    Prompt: {{prompt_text}}
    Answer:{{#select "answer" logprobs='logprobs'}} Yes{{or}} No{{/select}}'''
run_code(model,prompt,temperature)

Stop program Is the given following prompt AI generated? Answer with a single word, either "yes" or "no".
 Prompt: I remember the first time I saw the Face on Mars. It was a hazy summer day, and my colleague and I were discussing our latest research findings over a cup of coffee in the NASA control room. My colleague lowered his cup and pointed to the large screen television in the corner. "Look at that," he said, his eyes wide as saucers. 

I turned to see a grainy photograph of a rocky surface covered in craters and odd-shaped mounds. And then I saw it. In the middle of the frame, looming over the surrounding landscape, was a gigantic rock formation shaped like a human face, complete with a distinct jawline and a pair of hollow eyes. I nearly dropped my coffee cup. 

"That's the Face on Mars," my colleague said, sounding equally stunned. 

I had heard of the Face on Mars, of course, but I had never seen it up close like this. I leaned in for a closer look, and that's when I noticed something strange. The image was blurry and grainy, but as I looked closer, I could see that the "eyes" of the Face were actually just two sunken pits in the rock, not unlike the craters that covered the rest of the planet's surface. 

"It's a natural landform," I said, feeling a surge of excitement as the pieces of the puzzle began to fall into place. "The light and shadows must have created the illusion of a face." 

My colleague shook his head. "But what about the size of it? It's huge! Nothing natural could create a formation like that." 

I smiled. "Nature creates some pretty amazing things," I said. "But in this case, the shape of the rock, the lighting, and the shadows must have all combined to create the illusion of a face. It's just like the Nazca Lines in Peru." 

My colleague looked unconvinced. "But what about the eyes? They look like they were made by something else, something intelligent." 

I smiled. "Nature is intelligent," I said. "The eyes are just two pits in the rock that happen to be shaped in a way that resembles eyes. It's an amazing phenomenon, and it's a perfect example of how nature can create something that looks like it was made by something else." 

My colleague nodded slowly. "I see your point," he said. "But it's still a little bit freaky." 

I laughed. "It's all a matter of perspective," I said. "But the next time you see the Face on Mars, remember that it's just a natural landform, and there's no need to worry about aliens." 

I walked back to my desk, still feeling a bit amazed at the power of nature to create such an eerie phenomenon. But as I sat down and got back to work, I couldn't help but feel a sense of respect for the incredible diversity of the planet we call home. 
 Answer: Yes No No

In [8]:
file = open(output_file, "r")
data = file.readlines() 
predicted_labels=[]
for line in data:
    if(line.strip()=="No"):
        predicted_labels.append(0)
    else:
        predicted_labels.append(1)
file.close() 
print(len(predicted_labels))

1893


In [9]:
print(len(true_labels))

1893


In [10]:
from sklearn.metrics import precision_score, recall_score, f1_score
def calculate_metrics(true_labels, predicted_labels):
    precision = precision_score(true_labels, predicted_labels)
    recall = recall_score(true_labels, predicted_labels)
    f1 = f1_score(true_labels, predicted_labels)
    return precision, recall, f1
precision_value, recall_value, f1_score_value = calculate_metrics(true_labels, predicted_labels)
print(category,precision_value, recall_value, f1_score_value)

The Face on Mars 0.12321428571428572 0.22258064516129034 0.15862068965517243
